# MongoEngine

## 1. Setup

### 1.1. Imports

In [67]:
import mongoengine as engine

from mongoengine import (Document, IntField, LongField, StringField, ObjectIdField,
                         DateField, DateTimeField, ListField, ReferenceField)
from bson.objectid import ObjectId
from datetime import date, datetime as dt

### 1.2. Utils

In [ ]:
def format_array(elements):
    if not elements:
        return '[]'
    
    if len(elements) < 2:
        return '[{}]'.format(elements[0])
    
    output = ['[\n']
    for e in elements:
        output.append('{}{}\n'.format(' ' * 4, e))
    output.append(']')
    return ''.join(output)

## 2. Connect to MongoDB

### 2.1. MongoEngine connection

Arguments:
- `db`: which database will be connected
- `host`: mongo host name or ip address
- `username`: username for connected database
- `password`: password for connected username
- `alias`: name of this connection, for connection more difference hosts

In [ ]:
conn = engine.connect(db='test_db', host='localhost:27017', alias='test_db')
print('* db connectioned, connection is: {}'.format(conn))

### 2.2. Get PyMongo instance

In [ ]:
db = engine.get_db(alias='test_db')
print('* used db is: {}'.format(db))

## 3. Use MongoEngine

### 3.1. Document

#### 3.1.1. Define Document class

`Document` class define a collection, `meta` means metadata of collection
- `db_alias`: which connection should be used
- `collection`: name of collection which should be mapping to

- Define

In [ ]:
class User1(Document):
    meta = {
        'db_alias': 'test_db',
        'collection': 'user'
    }

    id_ = LongField(min_value=0, required=True, db_field='id', unique=True)
    name = StringField(max_length=100, required=True)
    gender = StringField(regex=r'[M|F]', max_length=1, required=True, default='M')
    
    def __str__(self):
        return 'User(id={}, id_={}, name="{}", gender="{}")'.format(self.id, self.id_, self.name, self.gender)

- Test

In [ ]:
try:

    # try to save normal Document object
    user = User1(id_=1, name='Alvin', gender='M')
    user.save()
    print('* user {} was created'.format(user))

    # save Document with default field gender
    user = User1(id_=2, name='Authur')
    user.save()
    print('* user {} was created'.format(user))

    # list all saved objects
    users = User1.objects.all()
    print('* there are {} objects created, and they are: {}'.format(len(users), format_array(users)))

    # try to trigger key duplicate error (by id_ field duplicated)
    try:
        user = User1(id_=2, name='Authur')
        user.save()
        print('* user {} was created'.format(user))
    except Exception as err:
        print('* error caused: {}'.format(err))

    # try to trigger validation error (by gender not match regex rule)
    try:
        user = User1(id_=3, name='Emma', gender='X')
        user.save()
    except Exception as err:
        print('* error caused: {}'.format(err))
finally:
    db.drop_collection('user')

#### 3.1.2. Id of Document

- `primary_key`: physics id of collection (_id property of collection)
- `id`: readonly value mappting to primary key (cannot define `id` property for Document explicit)
- `ObjectIdField`: object id type, required and primary key by default

##### 3.1.2.1. `ObjectIdField`

- Document define

In [ ]:
class User2(Document):
    meta = {
        'db_alias': 'test_db',
        'collection': 'user'
    }

    key = ObjectIdField() # key field, required and primary key by default
    name = StringField(max_length=100, required=True)
    gender = StringField(regex=r'[M|F]', max_length=1, required=True)

    def __str__(self):
        # self.id mappting to primary key (key field)
        return 'User(id={}, name="{}", gender="{}")'.format(self.id, self.name, self.gender)

- Test

In [ ]:
try:
    # ignore key property, use default ObjectId value

    for n in range(1, 4):
        user = User2(name='Alvin{}'.format(n), gender='M' if n % 2 == 0 else 'F')
        user.save()

    users = User2.objects.all()
    print('* there are {} objects created, and they are: {}'.format(len(users), format_array(users)))

    # use ObjectId object as value of key

    for n in range(1, 4):
        user = User2(key=ObjectId(), name='Alvin{}'.format(n), gender='M' if n % 2 == 0 else 'F')
        user.save()

    users = User2.objects.all()
    print('\n* there are {} objects created, and they are: {}'.format(len(users), format_array(users)))

    # use user define ObjectId as value of key

    for n in range(1, 4):
        user = User2(key=ObjectId('{}'.format(n) * 24), name='Alvin{}'.format(n), gender='M' if n % 2 == 0 else 'F')
        user.save()

    users = User2.objects.all()
    print('\n* there are {} objects created, and they are: {}'.format(len(users), format_array(users)))
finally:
    db.drop_collection('user')

##### 3.1.2.2. `primary_key`

- Document define

In [ ]:
class User3(Document):
    meta = {
        'db_alias': 'test_db',
        'collection': 'user'
    }

    key = StringField(max_length=10, min_length=2, required=True, primary_key=True) # primary key
    name = StringField(max_length=100, required=True)
    gender = StringField(regex=r'[M|F]', max_length=1, required=True)

    def __str__(self):
        # self.id mappting to primary key (key field)
        return 'User(id={}, name="{}", gender="{}")'.format(self.id, self.name, self.gender)

- Test

In [ ]:
try:
    for n in range(1, 4):
        user = User3(key='a' * (n + 2), name='Alvin{}'.format(n), gender='M' if n % 2 == 0 else 'F')
        user.save()

    users = User3.objects.all()
    print('* there are {} objects created, and they are: {}'.format(len(users), format_array(users)))
finally:
    db.drop_collection('user')

##### 3.1.2.3. `_id` field and primary key

- Document define 1

In [ ]:
class User4(Document):
    meta = {
        'db_alias': 'test_db',
        'collection': 'user'
    }

    # _id field means _id property of collection
    # if without 'primary_key', '_id' field cannot mapping to 'id' field
    _id = StringField(required=True)
    name = StringField(max_length=100, required=True)
    gender = StringField(regex=r'[M|F]', max_length=1, required=True)

    def __str__(self):
        return 'User(id={}, name="{}", gender="{}")'.format(self._id, self.name, self.gender)

- Test 1

In [ ]:
try:
    user = User4(_id='user_1', name='Alvin1', gender='M' if n % 2 == 0 else 'F')
    user.save()

    user = User4(_id='user_1', name='Alvin2', gender='M' if n % 2 == 0 else 'F')
    user.save()  # save document with same _id, means update collection by same _id

    users = User4.objects.all()
    print('* there are {} objects created, and they are: {}'.format(len(users), format_array(users)))
finally:
    db.drop_collection('user')

- Document define 2

In [ ]:
class User5(Document):
    meta = {
        'db_alias': 'test_db',
        'collection': 'user'
    }

    # if with 'primary_key', '_id' field can mapping to 'id' field
    _id = StringField(required=True, primary_key=True)
    name = StringField(max_length=100, required=True)
    gender = StringField(regex=r'[M|F]', max_length=1, required=True)

    def __str__(self):
        return 'User(id={}, name="{}", gender="{}")'.format(self.id, self.name, self.gender)

- Test 2

In [ ]:
try:
    user = User5(_id='user_1', name='Alvin1', gender='M' if n % 2 == 0 else 'F')
    user.save()

    user = User5(_id='user_1', name='Alvin2', gender='M' if n % 2 == 0 else 'F')
    user.save() # save document with same primary key, means update collection by same primary key

    users = User5.objects.all()
    print('* there are {} objects created, and they are: {}'.format(len(users), format_array(users)))
finally:
    db.drop_collection('user')

#### 3.1.3. Date & Datetime field

- Document define

In [ ]:
class User6(Document):
    meta = {
        'db_alias': 'test_db',
        'collection': 'user'
    }

    name = StringField(max_length=100, required=True)
    gender = StringField(regex=r'[M|F]', max_length=1, required=True)
    birthday = DateField(required=True)
    created_at = DateTimeField(required=True, default=dt.utcnow())

    def __str__(self):
        return 'User(id={}, name="{}", gender="{}", birthday={}, created_at={})'.format(
            self.id, self.name, self.gender, self.birthday, self.created_at)

- Test

In [ ]:
try:
    user = User6(name='Alvin', gender='M', birthday=date(1981, 3, 17))
    user.save()

    users = User6.objects.all()
    print('* there are {} objects created, and they are: {}'.format(len(users), format_array(users)))
finally:
    db.drop_collection('user')

#### 3.1.4. ListField field

- Document define

In [ ]:
class User7(Document):
    meta = {
        'db_alias': 'test_db',
        'collection': 'user'
    }

    name = StringField(max_length=100, required=True)
    gender = StringField(regex=r'[M|F]', max_length=1, required=True)
    hobbies = ListField(StringField(required=True), required=True)

    def __str__(self):
        return 'User(id={}, name="{}", gender="{}", hobbies={})'.format(self.id, self.name, self.gender, self.hobbies)

- Test

In [ ]:
try:
    user = User7(name='Alvin', gender='M', hobbies=['Football', 'Gaming'])
    user.save()

    users = User7.objects.all()
    print('* there are {} objects created, and they are: {}'.format(len(users), format_array(users)))

    # try to trigger validation error (list field include empty element)
    try:
        user = User7(name='Alvin', gender='M', hobbies=[])
        user.save()
    except Exception as err:
        print('* error caused: {}'.format(err))
finally:
    db.drop_collection('user')

#### 3.1.5. Reference field

- Document define

In [93]:
class Category(Document):
    meta = {
        'db_alias': 'test_db',
        'collection': 'category'
    }

    name = StringField(max_length=30, required=True)
    artnum = IntField(default=0, required=True)
    date = DateTimeField(required=True, default=dt.utcnow())

    def __str__(self):
        return 'Category(id={}, name="{}", artnum={}, date={})'.format(self.id, self.name, self.artnum, self.date)

class Post(Document):
    meta = {
        'db_alias': 'test_db',
        'collection': 'post'
    }

    title = StringField(max_length=100, required=True)
    content = StringField(required=True)
    tags = ListField(StringField(max_length=20, required=True), required=True)
    category = ReferenceField(Category)

    def __str__(self):
        return 'Post(id={}, title="{}", content={}, tags={}, \n\t category={})'.format(self.id, self.title, self.content,
                                                                                       self.tags, self.category)

- Test

In [95]:
try:
    # create some categories
    category1 = Category(name='Computer', artnum=1)
    category1.save()
    
    category2 = Category(name='Art', artnum=2)
    category2.save()

    categories = Category.objects.all()
    print('* there are {} objects created, and they are: {}'.format(len(categories), format_array(categories)))

    # create some posts, and each post reference category
    post = Post(title='Study mongodb', content='Study how to use mongodb', tags=['db', 'study'], category=category1)
    post.save()
    
    post = Post(title='Da Vinci', content='The story of Da Vinci', tags=['painting', 'artist'], category=category2)
    post.save()
    
    posts = Post.objects.all()
    print('* there are {} objects created, and they are: {}'.format(len(posts), format_array(posts)))
finally:
    db.drop_collection('category')
    db.drop_collection('post')

* there are 2 objects created, and they are: [
    Category(id=5e738b4b30231bba7fc82006, name="Computer", artnum=1, date=2020-03-19 15:03:37.430000)
    Category(id=5e738b4b30231bba7fc82007, name="Art", artnum=2, date=2020-03-19 15:03:37.430000)
]
* there are 2 objects created, and they are: [
    Post(id=5e738b4b30231bba7fc82008, title="Study mongodb", content=Study how to use mongodb, tags=['db', 'study'], 
	 category=Category(id=5e738b4b30231bba7fc82006, name="Computer", artnum=1, date=2020-03-19 15:03:37.430000))
    Post(id=5e738b4b30231bba7fc82009, title="Da Vinci", content=The story of Da Vinci, tags=['painting', 'artist'], 
	 category=Category(id=5e738b4b30231bba7fc82007, name="Art", artnum=2, date=2020-03-19 15:03:37.430000))
]


In [ ]:
engine.disconnect(alias='test_db')